In [1]:
import torch
import torch.nn as nn
import scipy.io
import torch.nn.functional as F

In [2]:
dd=torch.randn(10,1,3840) 

In [5]:
class CNN_1(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.5):
        super(CNN_1, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

        self.conv1_1 = nn.Conv1d(1, 8, kernel_size=8, stride=2, padding=3)#输入3840，输出1920
        self.conv1_2 = nn.Conv1d(1, 8, kernel_size=16, stride=2, padding=7)#输入3840，输出1920
        self.conv1_3 = nn.Conv1d(1, 8, kernel_size=32, stride=2, padding=15)#输入3840，输出1920
        self.conv1_4 = nn.Conv1d(1, 8, kernel_size=64, stride=2, padding=31)#输入3840，输出1920

        self.norm1 = nn.BatchNorm1d(32) # 1920   所以4*8=32个通道
        self.pool1 = nn.MaxPool1d(6, stride=4, padding=1, return_indices=True) # 480  所以1920/4=480

        self.conv2_1 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1) #尺寸不变
        self.conv2_2 = nn.Conv1d(32, 16, kernel_size=5, stride=1, padding=2) #尺寸不变

        self.norm2 = nn.BatchNorm1d(32)   #所以又16*2=32通道
        self.pool2 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 240


        self.conv3_1 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv1d(32, 16, kernel_size=5, stride=1, padding=2)

        self.norm3 = nn.BatchNorm1d(32)
        self.pool3 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 120 x 32  大小120，通道32

        self.fc = nn.Linear(120*32, hidden_size)

    def forward(self, x):
        # batch x 1 x raw_feature

        encoded = torch.cat([self.conv1_1(x),self.conv1_2(x),self.conv1_3(x),self.conv1_4(x)],1)
        encoded = self.relu(self.norm1(encoded))
        encoded, indices1 = self.pool1(encoded)

        encoded = torch.cat([self.conv2_1(encoded),self.conv2_2(encoded)],1)
        encoded = self.relu(self.norm2(encoded))
        encoded, indices2 = self.pool2(encoded)

        encoded = torch.cat([self.conv3_1(encoded),self.conv3_2(encoded)],1)
        encoded = self.relu(self.norm3(encoded))
        encoded, indices3 = self.pool3(encoded)

#         encoded = self.relu(self.fc(encoded.view(encoded.size(0),-1))) # batch x hidden_size
        return encoded
djgnet=CNN_1(hidden_size=128)

In [6]:
yy=djgnet(dd)
print(type(yy))
print(len(yy))
print(yy.shape)

<class 'torch.Tensor'>
10
torch.Size([10, 32, 120])
